
# Shared rides with pricing - Delft 

 ### Choice Function (Deterministic):
 `pool_price.py`
 * Pickup Distance: distance from driver initial position to the first pickup point
 * Travel Distance: distance from driver's initial position to the drop off point of the last passenger
 * Operating Cost: This include all the expenses
 * Profit: Driver revenue to serve the request
            
 
  ### KPI:
   
   * Profit of Individual driver
   * Profit of all the drivers
   * No.of rejected rides
   * U - PAX (Utility) 
  
   ### TBD- Choice Function (Probablistic):
  
  * choice logic to be applied inside `pool_price.py` 
  * P(R)= exp(beta * Profit_R)/ sum_all the rides( exp(beta * Profit_R)
 



In [1]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging

import pandas as pd
import ExMAS


## Pricing for Delft, Netherlands

In [3]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 0.1 # 6 minutes hour of simulation
params.nP = 8 # reuqests (and passengers)
params.nV = 8 # vehicles

## Parameters for ExMAS

In [4]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.3
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_pax' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True

inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 


03-02-23 10:15:12-INFO-Initializing pairwise trip shareability between 8 and 8 trips.
03-02-23 10:15:12-INFO-creating combinations
03-02-23 10:15:12-INFO-56	 nR*(nR-1)
03-02-23 10:15:12-INFO-Reduction of feasible pairs by 96.43%
03-02-23 10:15:12-INFO-Degree 2 	Completed
03-02-23 10:15:12-INFO-trips to extend at degree 2 : 9
03-02-23 10:15:12-INFO-At degree 2 feasible extensions found out of 0 searched
03-02-23 10:15:12-INFO-Degree 3 	Completed
03-02-23 10:15:12-INFO-No more trips to exted at degree 3


In [5]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

03-02-23 10:15:14-INFO-Matching 8 trips to 17 rides in order to min u_pax
03-02-23 10:15:14-INFO-Problem solution: Optimal. 
Total costs for single trips:             50 
reduced by matching to:                   45


In [7]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],7.42450,401,1,[7.4245],"[0, 401]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0,"[None, 2162556055, 3452839336]","[None, 0, 0]",node time req_id od 0 ...,401,4.01,6.02100,1.20420,4.81680
1,[1],6.58750,356,1,[6.5875],"[90, 356]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",0,"[None, 44730116, 44842366]","[None, 1, 1]",node time req_id od 0 NaN...,356,3.56,5.34150,1.06830,4.27320
2,[2],3.37450,182,1,[3.3745],"[101, 182]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]",1,"[None, 2612573901, 629532224]","[None, 2, 2]",node time req_id od 0 ...,182,1.82,2.73750,0.54750,2.19000
3,[3],4.09750,221,1,[4.0975],"[131, 221]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]",0,"[None, 4124549374, 44852053]","[None, 3, 3]",node time req_id od 0 ...,221,2.21,3.32400,0.66480,2.65920
4,[4],10.18400,550,1,[10.184000000000001],"[150, 550]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",0,"[None, 44858835, 446772563]","[None, 4, 4]",node time req_id od 0 N...,550,5.50,8.25900,1.65180,6.60720
5,[5],5.32950,288,1,[5.3295],"[168, 288]",[5],[5],1,5,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]",0,"[None, 44715422, 874115743]","[None, 5, 5]",node time req_id od 0 N...,288,2.88,4.32150,0.86430,3.45720
6,[6],5.73950,310,1,[5.7395],"[247, 310]",[6],[6],1,6,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0]",1,"[None, 1433962919, 44814383]","[None, 6, 6]",node time req_id od 0 ...,310,3.10,4.65450,0.93090,3.72360
7,[7],7.92550,428,1,[7.9255],"[299, 428]",[7],[7],1,7,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",0,"[None, 1833104239, 44859202]","[None, 7, 7]",node time req_id od 0 ...,428,4.28,6.42750,1.28550,5.14200
8,"[0, 4]",15.35590,758,20,"[6.940149999999999, 8.41575]","[-29.0, 208, 362, 188]","[0, 4]","[0, 4]",2,8,...,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]",1,"[None, 2162556055, 44858835, 3452839336, 44677...","[None, 0, 4, 0, 4]",node time req_id od 0 ...,758,7.58,9.99600,1.99920,7.99680
9,"[0, 7]",13.93735,909,20,"[7.0311499999999985, 6.906199999999999]","[-81.0, 461, 77, 371]","[0, 7]","[0, 7]",2,9,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]",0,"[None, 2162556055, 1833104239, 3452839336, 448...","[None, 0, 7, 0, 7]",node time req_id od 0 ...,909,9.09,8.71395,1.74279,6.97116


## Simulation with MaaSSim 

In [8]:
sim = simulate(params = params, inData = inData, logger_level = logging.WARNING) # simulate

03-02-23 10:15:27-INFO-Matching 8 trips to 17 rides in order to min u_pax
03-02-23 10:15:27-INFO-Problem solution: Optimal. 
Total costs for single trips:             50 
reduced by matching to:                   45
03-02-23 10:15:28-WARNING-Setting up 0.1h simulation at 2023-02-03 10:12:51 for 8 vehicles and 8 passengers in Delft, Netherlands
03-02-23 10:15:28-CRITICAL-this is request 0 with [0, 8, 9] available rides.
03-02-23 10:15:28-CRITICAL-ride 0 available [1]
03-02-23 10:15:28-CRITICAL-ride 8 available [1, 0]
03-02-23 10:15:28-CRITICAL-ride 9 available [1, 0]
03-02-23 10:15:28-CRITICAL-this is reuqest 0 with [0, 8, 9] still available rides.
03-02-23 10:15:28-CRITICAL-vehicle 1 has 3 choices
03-02-23 10:15:28-CRITICAL-vehicle 1 has chosen to serve request 0 with a ride 8 of degree 2, with travellers [0, 4].
03-02-23 10:15:28-CRITICAL-this is request 1 with [1, 10, 11] available rides.
03-02-23 10:15:28-CRITICAL-ride 1 available [1]
03-02-23 10:15:28-CRITICAL-ride 10 available [1,

In [10]:
pd.DataFrame(sim.vehs[1].myrides)


,veh,pos,t,event,paxes
0,1,1.436427e+09,0.0,STARTS_DAY,[]
1,1,1.436427e+09,0.0,OPENS_APP,[]
2,1,1.436427e+09,0.0,RECEIVES_REQUEST,[]
3,1,1.436427e+09,15.0,ACCEPTS_REQUEST,[]
4,1,1.436427e+09,35.0,IS_ACCEPTED_BY_TRAVELLER,[]
5,1,1.436427e+09,185.0,IS_ACCEPTED_BY_TRAVELLER,[]
6,1,2.162556e+09,302.0,ARRIVES_AT_PICKUP,[]
7,1,2.162556e+09,302.0,MEETS_TRAVELLER_AT_PICKUP,[]
8,1,2.162556e+09,332.0,DEPARTS_FROM_PICKUP,[0]
9,1,4.485884e+07,540.0,ARRIVES_AT_PICKUP,[0]


In [14]:
sim.res[0].pax_exp


,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,LOSES_PATIENCE,TRAVEL,WAIT,OPERATIONS
pax,,,,,,,,,,,,,,,,,
0,15.0,600.0,20.0,30.0,152.0,115.0,0.0,10.0,0,0,0,0,0,0,600.0,267.0,55.0
1,15.0,455.0,20.0,30.0,92.0,6.0,90.0,10.0,0,0,0,0,0,0,455.0,98.0,55.0
2,15.0,182.0,20.0,30.0,60.0,0.0,101.0,10.0,0,0,0,0,0,0,182.0,60.0,55.0
3,15.0,238.0,35.0,30.0,342.0,90.0,26.0,10.0,0,0,0,0,0,0,238.0,432.0,55.0
4,15.0,560.0,35.0,30.0,355.0,0.0,135.0,10.0,0,0,0,0,0,0,560.0,355.0,55.0
5,15.0,310.0,20.0,30.0,33.0,33.0,168.0,10.0,0,0,0,0,0,0,310.0,66.0,55.0
6,15.0,310.0,20.0,30.0,278.0,0.0,247.0,10.0,0,0,0,0,0,0,310.0,278.0,55.0
7,15.0,464.0,35.0,30.0,107.0,168.0,116.0,10.0,0,0,0,0,0,0,464.0,275.0,55.0


In [15]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM,REVENUE
veh,,,,,,,,,,,,,,,,,,,,,
1,2.0,0.0,False,0,0,0.0,15.0,0,170.0,0,...,0,0,6429.0,560.0,0,325.0,245.0,5299.0,9.520,9.520
2,0.0,0.0,False,0,0,0.0,0.0,0,0.0,0,...,0,0,7559.0,0.0,0,0.0,0.0,7559.0,0.000,0.000
3,0.0,0.0,False,0,0,150.0,0.0,0,0.0,0,...,0,0,7409.0,0.0,0,0.0,0.0,7409.0,0.000,0.000
4,2.0,0.0,False,0,0,90.0,15.0,0,61.0,0,...,0,0,6783.0,238.0,0,312.0,136.0,6097.0,5.777,5.777
5,2.0,0.0,False,0,0,168.0,15.0,0,88.0,0,...,0,0,6624.0,464.0,0,140.0,163.0,5857.0,7.166,7.166
6,1.0,0.0,False,0,0,101.0,15.0,0,20.0,0,...,0,0,7151.0,182.0,0,60.0,65.0,6844.0,1.825,1.825
7,1.0,0.0,False,0,0,247.0,15.0,0,20.0,0,...,0,0,6659.0,310.0,0,278.0,65.0,6006.0,3.103,3.103
8,0.0,0.0,False,0,0,299.0,0.0,0,0.0,0,...,0,0,7260.0,0.0,0,0.0,0.0,7260.0,0.000,0.000
